In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils

import altair as alt
import pandas as pd
import numpy as np
from vega_datasets import data

from constants import COLUMNS
from utils import apply_theme, get_visualization_subtitle
from web import for_website

In [ ]:
sites = pd.read_csv('../data/sites.csv', header=[0])
sites.head()

In [ ]:
countries = alt.topo_feature(data.world_110m.url, 'countries')

COUNTRIES = ["France", "Germany", "Italy", "Singapore", "USA"]
COUNTRY_ID = [250, 276, 380, 702, 840]
COUNTRY_COLOR = ["#0072B2", "#E69F00", "#009E73", "#CC79A7", "#D55E00"]
COLOR_BY_COUNTRY = {COUNTRIES[i]: COUNTRY_COLOR[i] for i in range(len(COUNTRIES))} 
COLOR_BY_ID = {COUNTRY_ID[i]: COUNTRY_COLOR[i] for i in range(len(COUNTRY_ID))} 
color_scale = alt.Scale(domain=COUNTRIES, range=COUNTRY_COLOR)
color_id_scale = alt.Scale(domain=COUNTRY_ID, range=COUNTRY_COLOR)


background = alt.Chart(countries).mark_geoshape(
    fill='lightgray',
    stroke='white'
).encode(
    color=alt.Color('id:N', scale=color_id_scale)
)

# city positions on background
locations = alt.Chart(sites).transform_aggregate(
    Latitude_Mean='mean(Latitude)',
    Longitude_Mean='mean(Longitude)',
    Hospitals_Total='sum(Hospitals)',
    Sites_Total='count()',
    groupby=['City','Country']
).mark_circle().encode(
    longitude='Longitude_Mean:Q',
    latitude='Latitude_Mean:Q',
    size=alt.Size( 'Hospitals_Total:Q', title='Number of Hospitals'),
    #size=alt.value(75),
    color=alt.Color('Country:N', scale=color_scale),
    tooltip=['City:N','Sites_Total:Q', 'Hospitals_Total:Q']
)

labels = locations.mark_text(
    align='center',
    baseline='top',
    dy=7
).encode(
    text='City',
    size=alt.value(10),
    color=alt.value('black')
)

In [ ]:
north_america_map = ( background + locations + labels ).project(
    type= 'mercator',
    scale= 380,                          # Magnify
    center= [-95,40],                     # [lon, lat]
    clipExtent= [[0, 0], [400, 300]],    # [[left, top], [right, bottom]]
).properties(
    title={
        "text": 'Sites in USA',
        "subtitle": get_visualization_subtitle(),
        "subtitleColor": "gray"
    },
    width=400, height=300,
)

apply_theme(north_america_map, legend_orient="right")

In [ ]:
europe_map = ( background + locations + labels ).project(
    type= 'mercator',
    scale=800,                          # Magnify
    center= [7,47],                     # [lon, lat]
    clipExtent= [[0, 0], [400, 300]],    # [[left, top], [right, bottom]]
).properties(
    title={
        "text": 'Sites in Europe',
        "subtitle": get_visualization_subtitle(),
        "subtitleColor": "gray"
    },
    width=400, height=300
)

apply_theme(europe_map, legend_orient="right")

In [ ]:
asia_map = ( background + locations + labels ).project(
    type= 'mercator',
    scale= 400,                          # Magnify
    center= [100,0],                     # [lon, lat]
    clipExtent= [[0, 0], [400, 300]],    # [[left, top], [right, bottom]]
).properties(
    title={
        "text": 'Sites in Asia',
        "subtitle": get_visualization_subtitle(),
        "subtitleColor": "gray"
    },
    width=400, height=300
)

apply_theme(asia_map, legend_orient="right")